In [2]:
pip install pymysql

  Obtaining dependency information for pymysql from https://files.pythonhosted.org/packages/e5/30/20467e39523d0cfc2b6227902d3687a16364307260c75e6a1cb4422b0c62/PyMySQL-1.1.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
   ---------------------------------------- 44.8/44.8 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pymysql
import pandas as pd

In [9]:
host_name = 'localhost'
host_port = 3306
username = 'admin'
password = 'pizza715'
database_name = 'mml'

db = pymysql.connect(
    host='mml.cu4cw1rqzfei.ap-northeast-2.rds.amazonaws.com',     # MySQL Server Address
    port=3306,     # MySQL Server Port
    user='admin',      # MySQL username
    passwd='pizza715',    # password for MySQL username
    db='mml',   # Database name
    charset='utf8'
)

SQL = "SHOW TABLES"
df = pd.read_sql(SQL, db)
df

C:\Users\AIschool\AppData\Local\Temp\ipykernel_20184\4087541180.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(SQL, db)


,Tables_in_mml
0,auth_group
1,auth_group_permissions
2,auth_permission
3,django_admin_log
4,django_content_type
5,django_migrations
6,django_session
7,mml_artist_gen
8,mml_music_info
9,mml_music_tag


In [11]:
SQL = 'SELECT * FROM mml_user_gen'
df = pd.read_sql(SQL, db)
df

C:\Users\AIschool\AppData\Local\Temp\ipykernel_20184\2548192682.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(SQL, db)


,id,priority,genre,username
0,10155,1,가요 / 댄스,00daK56D
1,10156,2,J-POP,00daK56D
2,10157,3,가요 / 랩/힙합,00daK56D
3,10158,4,가요 / 발라드,00daK56D
4,10159,5,가요 / R&B/소울,00daK56D
...,...,...,...,...
36197,46616,1,가요 / 랩/힙합,ZzwFsy2o
36198,46617,2,가요 / 댄스,ZzwFsy2o
36199,46618,3,J-POP,ZzwFsy2o
36200,46619,4,가요 / 발라드,ZzwFsy2o


In [14]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 점수 매핑을 위한 딕셔너리 생성
priority_scores = {1: 5, 2: 4, 3: 3, 4: 2, 5: 1}

# PRIORITY 컬럼에 따라 SCORE 컬럼 생성
df['score'] = df['priority'].map(priority_scores)

# 유저별 장르 점수 피벗 테이블 생성
user_genre_score_table = df.pivot_table(index='username', 
                                                   columns='genre', 
                                                   values='score', 
                                                   fill_value=0, 
                                                   aggfunc='sum')

# 코사인 유사도 계산
cosine_sim = cosine_similarity(user_genre_score_table)


# 유사도 행렬을 DataFrame으로 변환
cosine_sim_df = pd.DataFrame(cosine_sim, 
                             index=user_genre_score_table.index, 
                             columns=user_genre_score_table.index)


In [15]:
cosine_sim_df

username,00GV0hH5,00ZDycPL,00daK56D,012crtJW,01o3BSIy,01xFjnaC,02FoMC0v,02Gerxl3,03tU8EOK,052oK12A,...,zvOtBo2O,zwD8v536,zxVmsKdg,zxoHddtP,zyFQvWMy,zyYT7Km7,zylFjfo8,zypJJyvw,zzBYtLo7,zzq2i5A7
username,,,,,,,,,,,,,,,,,,,,,
00GV0hH5,1.000000,0.781818,0.654545,0.581818,0.654545,0.690909,0.654545,0.927273,0.945455,0.690909,...,0.727273,0.509091,0.618182,0.527273,0.945455,0.654545,0.672727,0.436364,0.272727,0.509091
00ZDycPL,0.781818,1.000000,0.800000,0.472727,0.563636,0.836364,0.600000,0.745455,0.781818,0.472727,...,0.709091,0.436364,0.418182,0.745455,0.836364,0.563636,0.654545,0.636364,0.381818,0.563636
00daK56D,0.654545,0.800000,1.000000,0.200000,0.236364,0.763636,0.527273,0.709091,0.545455,0.200000,...,0.963636,0.472727,0.236364,0.418182,0.563636,0.581818,0.800000,0.363636,0.418182,0.909091
012crtJW,0.581818,0.472727,0.200000,1.000000,0.763636,0.309091,0.509091,0.581818,0.690909,0.781818,...,0.218182,0.527273,0.581818,0.709091,0.709091,0.727273,0.490909,0.763636,0.527273,0.181818
01o3BSIy,0.654545,0.563636,0.236364,0.763636,1.000000,0.436364,0.254545,0.636364,0.709091,0.945455,...,0.254545,0.636364,0.654545,0.709091,0.763636,0.490909,0.527273,0.527273,0.436364,0.181818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zyYT7Km7,0.654545,0.563636,0.581818,0.727273,0.490909,0.600000,0.581818,0.836364,0.654545,0.509091,...,0.654545,0.309091,0.436364,0.600000,0.654545,1.000000,0.818182,0.636364,0.727273,0.654545
zylFjfo8,0.672727,0.654545,0.800000,0.490909,0.527273,0.581818,0.381818,0.854545,0.636364,0.527273,...,0.836364,0.618182,0.454545,0.454545,0.636364,0.818182,1.000000,0.345455,0.545455,0.872727
zypJJyvw,0.436364,0.636364,0.363636,0.763636,0.527273,0.600000,0.654545,0.436364,0.472727,0.400000,...,0.290909,0.200000,0.290909,0.927273,0.545455,0.636364,0.345455,1.000000,0.690909,0.218182


In [16]:
SQL = 'SELECT * FROM mml_user_his'
df1 = pd.read_sql(SQL, db)
df1

C:\Users\AIschool\AppData\Local\Temp\ipykernel_20184\616001107.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql(SQL, db)


,id,user,title,artist,genre,playtime,created_at
0,1,00daK56D,Back Porch (Live),Presidents Of The United States Of America,POP / 락,306,2021-02-12 15:09:00
1,2,00daK56D,Beethoven: Symphony No.3 In E Flat Major Op.55...,Various Artists,클래식,93,2021-03-23 21:53:00
2,3,00daK56D,"Camp 30 (Feat. Lexy, 조원선)",윤희중,가요 / 랩/힙합,143,2021-03-24 21:10:00
3,4,00daK56D,Comme On Se Retrouve,Karna Zoo,POP / 랩/힙합,260,2021-01-15 20:47:00
4,5,00daK56D,Don't Change Your Plans,Ben Folds Five,POP / 락,173,2021-01-09 17:40:00
...,...,...,...,...,...,...,...
173143,173144,zZOtHLJ6,카페인 (Under Caffeine),스텔라장 (Stella Jang),가요 / 인디,199,2022-02-22 17:18:00
173144,173145,zzq2i5A7,Steps In The Sand,Wayne Gratz,뉴에이지,31,2021-08-13 22:55:00
173145,173146,ZzwFsy2o,"Knock 'Em Dead, Kid",Motley Crue,POP / 팝,71,2021-01-19 19:13:00
173146,173147,ZzwFsy2o,This One's For The Children,New Kids On The Block,POP / 락,53,2021-01-19 19:16:00


In [32]:
def get_top_10_similar_users_listening(user, df1, cosine_sim_df):
    # 입력받은 사용자와 가장 유사도가 높은 사용자들 찾기 (충분히 큰 숫자로 범위 확장)
    similar_users = cosine_sim_df[user].sort_values(ascending=False).index.tolist()
    similar_users.remove(user)  # 입력받은 사용자 제거

    # 유사한 사용자들의 청취 목록 찾기
    listening_lists = []
    for similar_user in similar_users:
        if len(listening_lists) < 10:
            user_listening = df1[df1['user'] == similar_user]
            if not user_listening.empty:
                listening_lists.append(user_listening)
        else:
            break

    return pd.concat(listening_lists) if listening_lists else pd.DataFrame()  # 청취 목록 반환

# 예제 사용자 ID (임의의 사용자 ID를 여기에 입력)
user = '052oK12A'  # 이 부분을 실제 사용자 ID로 변경

# 유사한 사용자들의 청취 목록 가져오기
top_10_similar_users_listening = get_top_10_similar_users_listening(user, df1, cosine_sim_df)

# 결과 출력
print(top_10_similar_users_listening)


            id      user              title         artist       genre  \
72978    72979  hUB5aI1v          Hound Dog  Elvis Presley     POP / 락   
72979    72980  hUB5aI1v  Love Song Tonight            양진석    가요 / 발라드   
72980    72981  hUB5aI1v     Zeness Mon Pei            Oze  POP / 랩/힙합   
72981    72982  hUB5aI1v              한번만 더            박성신    가요 / 발라드   
101986  101987  NBJEo7dr  Celui Qui S'en Va          Damia  POP / 월드뮤직   
...        ...       ...                ...            ...         ...   
88555    88556  kwajeD6s             설렘 가득히            박서현        뉴에이지   
88556    88557  kwajeD6s            신사동 그사람            주현미    가요 / 트로트   
88557    88558  kwajeD6s            여린 눈빛으로          모카피아노        뉴에이지   
88558    88559  kwajeD6s            천사의 목소리           하늘정원        뉴에이지   
88559    88560  kwajeD6s         행복한 추억의 시간            엄지은        뉴에이지   

        playtime          created_at  
72978        174 2021-02-28 10:47:00  
72979        179 2021-02-28 11:12

In [33]:
def get_top_20_unique_songs_by_artist_and_genre(user, user_genre_score_table, listening_lists):
    # 예제 사용자의 장르 선호도 순위 가져오기
    user_preferences = user_genre_score_table.loc[user].sort_values(ascending=False)

    # 청취 목록에서 선호 장르에 해당하는 곡들 찾기
    preferred_songs = listening_lists[listening_lists['genre'].isin(user_preferences.index)]

    # 장르 선호도를 기준으로 곡들을 정렬
    preferred_songs['genre_preference_score'] = preferred_songs['genre'].map(user_preferences)
    sorted_songs = preferred_songs.sort_values(by='genre_preference_score', ascending=False)

    # 아티스트와 장르가 겹치는 곡들 중 하나만 남기고 상위 20곡 선택
    top_20_unique_songs = sorted_songs.drop_duplicates(subset=['artist', 'genre']).head(20)

    return top_20_unique_songs

# 예제 사용자의 아티스트와 장르가 겹치는 곡 중 하나만 남기고 상위 20곡 추출
top_20_unique_songs_by_artist_and_genre = get_top_20_unique_songs_by_artist_and_genre(user, user_genre_score_table, top_10_similar_users_listening)

# 결과 출력
print(top_20_unique_songs_by_artist_and_genre)


            id      user                                  title  \
72979    72980  hUB5aI1v                      Love Song Tonight   
161457  161458  Yd70GinN                             니가 있는 세상에서   
102012  102013  NBJEo7dr                                     마리   
102017  102018  NBJEo7dr                                새장 속의 나   
102019  102020  NBJEo7dr                                   어머님께   
83267    83268  kejyVM6o                                 Memory   
72981    72982  hUB5aI1v                                  한번만 더   
161456  161457  Yd70GinN                                    긴 잠   
161461  161462  Yd70GinN                                 행복한 나무   
88534    88535  kwajeD6s                        Reason's To One   
101991  101992  NBJEo7dr                             Good Night   
102009  102010  NBJEo7dr                              너 그럴때면...   
111304  111305  OXz9lwfd                          Pray Of Angel   
88551    88552  kwajeD6s             나랑 산책할래요? (Vietato Fumare

C:\Users\AIschool\AppData\Local\Temp\ipykernel_20184\996426941.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preferred_songs['genre_preference_score'] = preferred_songs['genre'].map(user_preferences)
